In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import when,lit,col,split,expr
from pyspark.sql.types import StringType
import re

In [2]:
#start spark session
spark = SparkSession \
.builder \
.appName("Python Spark create RDD example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m


23/01/06 00:06:30 WARN Utils: Your hostname, georgemicha-premierleag-b6ljki953qu resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
23/01/06 00:06:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 00:06:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
def get_table():

    #EXTRACT with beautiful soup

    url = 'https://www.bbc.com/sport/football/premier-league/table'
    headers = []
    page = requests.get(url)
    soup = BeautifulSoup(page.text,  "html.parser")
    table= soup.find("table", class_="ssrcss-14j0ip6-Table e3bga5w4")

    for i in table.find_all('th'):
        title = i.text
        headers.append(title)
    league_table = pd.DataFrame(columns = headers)
    for j in table.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(league_table)
        league_table.loc[length] = row

    #TRANSFORM with PySpark

    league = spark.createDataFrame(league_table) 
    #drop unwanted column
    league = league.drop("Form, Last 6 games, Oldest first") 
    #rename columns
    league = league.withColumnRenamed("Goals For","Goals_For").withColumnRenamed("Goals Against","Goals_Against") \
                    .withColumnRenamed("Goal Difference","Goal_Difference")
    #add new columns based on existing columns
    league = league.withColumn("grade_class",  when((league.Lost == 0) & (league.Drawn == 0) \
                        & (league.Points <= league['Goals_For']), lit("S"))
                        .when((league.Lost == 0) & (league.Drawn <= 2), lit("A+"))
                        .when((league.Lost <= 1) & (league.Drawn <= 3), lit("A"))
                    .when((league.Lost <= 1  ) & (league.Drawn <= 4  ), lit('B'))
                    .otherwise('C')) 

    league = league.withColumn('conqueror', lit(league.Points - (league.Played * 3)).cast('integer'))
    # conqueror column, the difference between the teams current points and the points if they didn't loose or draw any math
    league.show()



## Top Scorers table

In [46]:
url = 'https://www.bbc.com/sport/football/premier-league/top-scorers'
headers = []
page = requests.get(url)
soup = BeautifulSoup(page.text,  "html.parser")
table= soup.find("table", class_="gs-o-table")

for i in table.find_all('th'):
    title = i.text
    headers.append(title)
top_scorers = pd.DataFrame(columns = headers)
for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    length = len(top_scorers)
    top_scorers.loc[length] = row



top_scorers 


,Rank,Name,GoalsGL,AssistsA,PlayedP,Goals per 90,Mins per GoalMPG,Total Shots,Goal Conversion,Shot Accuracy
0,1,Erling HaalandHaalandManchester CityMan City,21,3,16,1.44,62,55,38%,62%
1,2,Harry KaneKaneTottenham HotspurTottenham,15,1,18,0.84,107,48,31%,69%
2,3,Ivan ToneyToneyBrentfordBrentford,12,3,16,0.75,120,41,29%,54%
3,4,Aleksandar MitrovicMitrovicFulhamFulham,11,1,15,0.77,117,55,20%,51%
4,5,Rodrigo MorenoRodrigoLeeds UnitedLeeds,10,1,16,0.79,114,32,31%,56%
5,6,Miguel AlmirónAlmirónNewcastle UnitedNewcastle,9,1,18,0.54,167,25,36%,56%
6,7,Martin ØdegaardØdegaardArsenalArsenal,7,5,16,0.47,193,26,27%,58%
7,7,James MaddisonMaddisonLeicester CityLeicester,7,4,13,0.57,157,25,28%,52%
8,7,Mohamed SalahSalahLiverpoolLiverpool,7,4,17,0.42,215,47,15%,51%
9,7,Roberto FirminoFirminoLiverpoolLiverpool,7,3,13,0.67,134,26,27%,77%
